In [2]:
import re

def extract_sql_statement(text):
    sql_pattern = re.compile(r"(SELECT|INSERT|UPDATE|DELETE|CREATE|ALTER|DROP|GRANT|REVOKE|TRUNCATE|MERGE|CALL|EXPLAIN|SHOW|USE)\b.*?;", re.IGNORECASE | re.DOTALL)
    match = sql_pattern.search(text)
    # If a match is found, return the matched string
    if match:
        return match.group(0)
    else:
        return None

extracted_sql = extract_sql_statement(text_with_sql)
print(extracted_sql)


SELECT 
    "order"."id" AS "case_id", 
    "AA"."id" AS "activity_id", 
    "AA"."creation_date" AS "timestamp"
FROM 
    "order"
JOIN 
    "invoice" ON "order"."id" = "invoice"."order_id"
JOIN 
    "payment" ON "invoice"."id" = "payment"."invoice_id"
JOIN 
    "shipment" ON "order"."id" = "shipment"."order_id"
JOIN 
    "AA" ON "AA"."id" = "order"."id"
UNION ALL
SELECT 
    "order"."id", 
    "invoice"."id", 
    "invoice"."creation_date"
FROM 
    "order"
JOIN 
    "invoice" ON "order"."id" = "invoice"."order_id"
UNION ALL
SELECT 
    "order"."id", 
    "payment"."id", 
    "payment"."creation_date"
FROM 
    "order"
JOIN 
    "invoice" ON "order"."id" = "invoice"."order_id"
JOIN 
    "payment" ON "invoice"."id" = "payment"."invoice_id"
UNION ALL
SELECT 
    "order"."id", 
    "shipment"."id", 
    "shipment"."creation_date"
FROM 
    "order"
JOIN 
    "shipment" ON "order"."id" = "shipment"."order_id"
ORDER BY 
    "timestamp";


In [1]:
from utils.apps import SimpleApp
from langchain_openai import ChatOpenAI
from utils.db import get_database_schema_execute_all
from utils.helper import save_results
import os
from keys import OPENAI_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_KEY

In [2]:
# Collect all dbs and all prompts
db_list = [x for x in os.listdir('testDBs') if '.' not in x]
prompt_list = [x for x in os.listdir('prompts') if '.txt' in x]

print(db_list[0:])
# print(prompt_list[0:])

# LLM model and parameters
llm_model = ChatOpenAI(model="gpt-4", temperature=0)

# Loop over all dbs and prompt combinations
for db in db_list[0:]:

    path_to_csv_files = 'testDBs/'+db+'/db/'
    path_to_csv_schema_file = 'testDBs/'+db+'/csv_schema.xlsx'
    path_to_groud_truth_eventlog = 'testDBs/'+db+'/ground-truth-eventlog.csv'
    db_output_dir = 'testDBs/'+db+'/example.db'
    result_output_dir = 'testDBs/'+db+'/results/'
    db_schema = get_database_schema_execute_all(path_to_csv_files = path_to_csv_files,path_to_csv_schema_file=path_to_csv_schema_file, db_output_dir= db_output_dir)

    for pr in prompt_list[0:]:
        prompt_file_name = pr.split('.txt')[0]
        file = open('prompts/'+pr, "r")
        file_txt = file.read()
        file.close()

        prompt=(f"""Consider the following db schema: {db_schema}"""+file_txt
               +f"""Write one sql statement that returns an event log from the selected tables with the following columns: case_id,activity_id,timestamp. """
               +f"""Use quotes for identifiers. Provide only the query.""")
        # print(prompt)

        AgentState = {"messages": [prompt]}
        a = SimpleApp(path_to_db = db_output_dir, path_to_groud_truth_eventlog=path_to_groud_truth_eventlog, llm_model = llm_model)
        result = a.invoke(AgentState)
        save_results(chain_response=result, output_dir=result_output_dir, prompt_file=prompt_file_name)
        # print('\n EXECUTED SQL STATEMNENT: \n', result['agent_response'])
        # print(result)
        try:
            print([db, pr ,result['result']])
        except:
            print([db,pr,f"""Can't calculate Precision, Recall and F1. ERROR in SQL"""])       

['test1', 'UWV_case_1', 'baseline-with-misleading-table', 'baseline-with-useless-table', 'baseline (aka toy database)', 'BPI2016-V31', 'RunningExample-V03', 'RunningExample-V15', 'RunningExample-V63', 'UWV-V06', 'UWV-V22', 'UWV-V31', 'BPI2016-V07', 'BPI2016-V24', 'paperj-V1', 'paperj-V4', 'paperj-V6']
CREATE TABLE "order" ("id" TEXT, "creation_date" DATETIME, PRIMARY KEY ("id"))
CREATE TABLE "payment" ("id" TEXT, "invoice_id" TEXT, "creation_date" DATETIME, PRIMARY KEY ("id"), FOREIGN KEY ("invoice_id") REFERENCES "invoice"("id"))
CREATE TABLE "shipment" ("id" TEXT, "order_id" TEXT, "creation_date" DATETIME, PRIMARY KEY ("id"), FOREIGN KEY ("order_id") REFERENCES "order"("id"))
CREATE TABLE "invoice" ("id" TEXT, "order_id" TEXT, "creation_date" DATETIME, PRIMARY KEY ("id"), FOREIGN KEY ("order_id") REFERENCES "order"("id"))
CREATE TABLE "AA" ("id" TEXT, "a" TEXT, "b" TEXT)
Data inserted into table order
Data inserted into table invoice
Data inserted into table payment
Data inserted int

Skipping write for channel tags which has no readers
Skipping write for channel tags which has no readers
Skipping write for channel tags which has no readers


['test1', 'prompt-baseline.txt', {'precision': 0.0, 'recall': 0.0, 'f1': 0, 'relaxed_precision': 0.692, 'relaxed_recall': 1.0, 'relaxed_f1': 0.818}]


Skipping write for channel tags which has no readers
Skipping write for channel tags which has no readers
Skipping write for channel tags which has no readers


['test1', 'prompt-chain-of-thought.txt', {'precision': 0.0, 'recall': 0.0, 'f1': 0, 'relaxed_precision': 0.462, 'relaxed_recall': 0.462, 'relaxed_f1': 0.462}]


...

Skipping write for channel tags which has no readers
Skipping write for channel tags which has no readers
Skipping write for channel tags which has no readers


['BPI2016-V31', 'prompt-tree-of-thought_persona.txt', {'precision': 0.007, 'recall': 0.007, 'f1': 0.007, 'relaxed_precision': 0.671, 'relaxed_recall': 0.671, 'relaxed_f1': 0.671}]


Skipping write for channel tags which has no readers
Skipping write for channel tags which has no readers
Skipping write for channel tags which has no readers


['BPI2016-V31', 'prompt-04-tree-of-thought.txt', {'precision': 0.007, 'recall': 0.007, 'f1': 0.007, 'relaxed_precision': 0.671, 'relaxed_recall': 0.671, 'relaxed_f1': 0.671}]


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## TEST 1 Toy Example

In [9]:
## TEST 1

# Change path to csv files and prompt
path_to_csv_files = 'testDBs/test1/db/'
path_to_csv_schema_file = 'testDBs/test1/csv_schema.xlsx'
path_to_groud_truth_eventlog = 'testDBs/test1/ground-truth-eventlog.csv'
db_output_dir = 'testDBs/test1/example.db'
db_schema = get_database_schema_execute_all(path_to_csv_files = path_to_csv_files,path_to_csv_schema_file=path_to_csv_schema_file, db_output_dir= db_output_dir)

CREATE TABLE "invoice" ("id" TEXT, "order_id" TEXT, "creation_date" DATETIME, PRIMARY KEY ("id"), FOREIGN KEY ("order_id") REFERENCES "order"("id"))
CREATE TABLE "order" ("id" TEXT, "creation_date" DATETIME, PRIMARY KEY ("id"))
CREATE TABLE "payment" ("id" TEXT, "invoice_id" TEXT, "creation_date" DATETIME, PRIMARY KEY ("id"), FOREIGN KEY ("invoice_id") REFERENCES "invoice"("id"))
CREATE TABLE "shipment" ("id" TEXT, "order_id" TEXT, "creation_date" DATETIME, PRIMARY KEY ("id"), FOREIGN KEY ("order_id") REFERENCES "order"("id"))
Data inserted into table order
Data inserted into table invoice
Data inserted into table payment
Data inserted into table shipment
Database created: testDBs/test1/example.db


In [10]:

# llm_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# llm_model = ChatOpenAI(model="gpt-4-turbo", temperature=0)
llm_model = ChatOpenAI(model="gpt-4", temperature=0)

            # Combine the payment table with the invoice tabel to derive the order for which the payment is made. Use the derived order_id as case_id for this part of the query.
            # Add this result to the records from the order table, invoice table and shipment table. Use the attribute in each table that is related to the id of the order as the case_id.
            # Use the name of the table where a record originates from as the activity_id.

prompt=f"""Consider the following db schema:
            {db_schema}
            Each record in each table represents at least one event.
            If a table contains multiple timestamp attributes, then each of these attributes is related to an event with the activity_id equal to the attribute name.
            
            Use the id of the order as the case notion.
            
            When a table does not have an attribute that contains the selected case notion, combine the necessary tables to obtain this case notion.
            
            Write one sql statement that returns an event log from the selected tables with the following columns: case_id,activity_id,timestamp. 
            In activity_id leave out "_created".
            Use quotes for identifiers. Provide only the query."""

# prompt=f"""Consider the relational database structure of the provided tables. 
#             Also consider you have a process model represented by start -> Create order -> Create invoice -> Deliver goods -> end. 
#             Where Create order, Create invoice, and Deliver goods are activity names of the process model, 
#             and "X -> Y" represents that any activity X is directly followed by an activity Y. 
#             With this in mind, now you need to write one sql statement that returns an event log from the tables with the following columns: case_id,activity_id,timestamp. 
#             Use quotes for identifiers. Provide only the query."""

In [11]:
# db_schema
prompt

'Consider the following db schema:\n            TABLE invoice:\nCREATE TABLE "invoice" ("id" TEXT, "order_id" TEXT, "creation_date" DATETIME, PRIMARY KEY ("id"), FOREIGN KEY ("order_id") REFERENCES "order"("id"));\n\nTABLE order:\nCREATE TABLE "order" ("id" TEXT, "creation_date" DATETIME, PRIMARY KEY ("id"));\n\nTABLE payment:\nCREATE TABLE "payment" ("id" TEXT, "invoice_id" TEXT, "creation_date" DATETIME, PRIMARY KEY ("id"), FOREIGN KEY ("invoice_id") REFERENCES "invoice"("id"));\n\nTABLE shipment:\nCREATE TABLE "shipment" ("id" TEXT, "order_id" TEXT, "creation_date" DATETIME, PRIMARY KEY ("id"), FOREIGN KEY ("order_id") REFERENCES "order"("id"));\n\n            Each record in each table represents at least one event.\n            If a table contains multiple timestamp attributes, then each of these attributes is related to an event with the activity_id equal to the attribute name.\n            \n            Use the id of the order as the case notion.\n            \n            When a

In [12]:
AgentState = {"messages": [prompt]}
a = SimpleApp(path_to_db = db_output_dir, path_to_groud_truth_eventlog=path_to_groud_truth_eventlog, llm_model = llm_model)
result = a.invoke(AgentState)
result

Skipping write for channel tags which has no readers
Skipping write for channel tags which has no readers
Skipping write for channel tags which has no readers


{'messages': ['Consider the following db schema:\n            TABLE invoice:\nCREATE TABLE "invoice" ("id" TEXT, "order_id" TEXT, "creation_date" DATETIME, PRIMARY KEY ("id"), FOREIGN KEY ("order_id") REFERENCES "order"("id"));\n\nTABLE order:\nCREATE TABLE "order" ("id" TEXT, "creation_date" DATETIME, PRIMARY KEY ("id"));\n\nTABLE payment:\nCREATE TABLE "payment" ("id" TEXT, "invoice_id" TEXT, "creation_date" DATETIME, PRIMARY KEY ("id"), FOREIGN KEY ("invoice_id") REFERENCES "invoice"("id"));\n\nTABLE shipment:\nCREATE TABLE "shipment" ("id" TEXT, "order_id" TEXT, "creation_date" DATETIME, PRIMARY KEY ("id"), FOREIGN KEY ("order_id") REFERENCES "order"("id"));\n\n            Each record in each table represents at least one event.\n            If a table contains multiple timestamp attributes, then each of these attributes is related to an event with the activity_id equal to the attribute name.\n            \n            Use the id of the order as the case notion.\n            \n    

In [ ]:
# SELECT 
#     "order"."id" AS "case_id", 
#     'order' AS "activity_id", 
#     "order"."creation_date" AS "timestamp"
# FROM "order"

# UNION ALL

# SELECT 
#     "invoice"."order_id" AS "case_id", 
#     'invoice' AS "activity_id",
#     "invoice"."creation_date" AS "timestamp"
# FROM "invoice"

# UNION ALL

# SELECT 
#     "payment"."id" AS "case_id", 
#     'payment' AS "activity_id", 
#     "payment"."creation_date" AS "timestamp"
# FROM 
#     "payment"
#     JOIN "invoice" ON "payment"."invoice_id" = "invoice"."id"

# UNION ALL

# SELECT 
#     "shipment"."order_id" AS "case_id",
#     'shipment' AS "activity_id",
#     "shipment"."creation_date" AS "timestamp"
# FROM "shipment";
 

## TEST UWV_case_1

In [14]:
# Change path to csv files and prompt
path_to_csv_files = 'testDBs/UWV_case_1/db/'
path_to_csv_schema_file = 'testDBs/UWV_case_1/csv_schema.xlsx'
path_to_groud_truth_eventlog = 'testDBs/UWV_case_1/AutoDrive_Result_20201515_Sample.csv'
db_output_dir = 'testDBs/UWV_case_1/example.db'
db_schema = get_database_schema_execute_all(path_to_csv_files = path_to_csv_files,path_to_csv_schema_file=path_to_csv_schema_file, db_output_dir= db_output_dir)

CREATE TABLE "AutoDrive_Event_100_20201215_Sample" ("id" TEXT, "EventDateTime" DATETIME, "ActivityID" TEXT, "Attribute_101" TEXT, PRIMARY KEY ("id", "EventDateTime", "ActivityID", "Attribute_101"))
CREATE TABLE "AutoDrive_Event_200_20201215_Sample" ("id" TEXT, "EventDateTime" DATETIME, "ActivityID" TEXT, PRIMARY KEY ("id", "EventDateTime", "ActivityID"))
CREATE TABLE "AutoDrive_Event_300_20201215_Sample" ("id" TEXT, "EventDateTime" DATETIME, "ActivityID" TEXT, "Attribute_301" TEXT, PRIMARY KEY ("id"))
CREATE TABLE "AutoDrive_Event_400_20201215_Sample" ("id" TEXT, "EventDateTime" DATETIME, PRIMARY KEY ("id"))
CREATE TABLE "AutoDrive_Event_500_20201215_Sample" ("session_id" TEXT, "id" TEXT, "ActivityID_501" DATETIME, "ActivityID_502" DATETIME, "ActivityID_503" DATETIME, "ActivityID_504" DATETIME, "ActivityID_505" DATETIME, "ActivityID_506" DATETIME, "ActivityID_507" DATETIME, "Attribute_510" TEXT, PRIMARY KEY ("id"))
Data inserted into table AutoDrive_Event_300_20201215_Sample
Data inser

In [18]:
# llm_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# llm_model = ChatOpenAI(model="gpt-4-turbo", temperature=0)
llm_model = ChatOpenAI(model="gpt-4", temperature=0)

            # Combine the payment table with the invoice tabel to derive the order for which the payment is made. Use the derived order_id as case_id for this part of the query.
            # Add this result to the records from the order table, invoice table and shipment table. Use the attribute in each table that is related to the id of the order as the case_id.
            # Use the name of the table where a record originates from as the activity_id.

prompt=f"""Consider the following db schema:
            {db_schema}
            Each record in each table represents at least one event.
            If a table contains multiple timestamp attributes, then each of these attributes is related to an event.
            Use the name of the column as the default value of the activity_id.
            
            Use CaseID as the case notion.
            When a table does not have an attribute that contains the selected case notion, combine the necessary tables to obtain this case notion.

            Write one sql statement that returns an event log from the selected tables with the following columns: case_id,activity_id,timestamp. 
            Use quotes for identifiers. 
            Use for all timestamps the most detailed timeformat that occurs in the tables.
            If a value in the table is empty or NULL leave out the whole record.
            Provide only the SQL query, no other comments should be part of the response."""

AgentState = {"messages": [prompt]}
a = SimpleApp(path_to_db = db_output_dir, path_to_groud_truth_eventlog=path_to_groud_truth_eventlog, llm_model = llm_model)
result = a.invoke(AgentState)
result

Skipping write for channel tags which has no readers
Skipping write for channel tags which has no readers
Skipping write for channel tags which has no readers


{'messages': ['Consider the following db schema:\n            TABLE AutoDrive_Event_100_20201215_Sample:\nCREATE TABLE "AutoDrive_Event_100_20201215_Sample" ("id" TEXT, "EventDateTime" DATETIME, "ActivityID" TEXT, "Attribute_101" TEXT, PRIMARY KEY ("id", "EventDateTime", "ActivityID", "Attribute_101"));\n\nTABLE AutoDrive_Event_200_20201215_Sample:\nCREATE TABLE "AutoDrive_Event_200_20201215_Sample" ("id" TEXT, "EventDateTime" DATETIME, "ActivityID" TEXT, PRIMARY KEY ("id", "EventDateTime", "ActivityID"));\n\nTABLE AutoDrive_Event_300_20201215_Sample:\nCREATE TABLE "AutoDrive_Event_300_20201215_Sample" ("id" TEXT, "EventDateTime" DATETIME, "ActivityID" TEXT, "Attribute_301" TEXT, PRIMARY KEY ("id"));\n\nTABLE AutoDrive_Event_400_20201215_Sample:\nCREATE TABLE "AutoDrive_Event_400_20201215_Sample" ("id" TEXT, "EventDateTime" DATETIME, PRIMARY KEY ("id"));\n\nTABLE AutoDrive_Event_500_20201215_Sample:\nCREATE TABLE "AutoDrive_Event_500_20201215_Sample" ("session_id" TEXT, "id" TEXT, "Act